### Talk Time Data Prepare

In [103]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [104]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [105]:
# language pair
lang_folder = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

# word usage
min_word_use_num = 1
max_word_use_num = 6

# youtube
sample_num = 1  # 6
time_shift = 0.6

In [106]:
def word_in_wordgroup_simple(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column row result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"search_string",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [107]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [108]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")\n
    df_search_result is dataframe and "search_string", "start_time", "end_time", "sentence", "video_id" are its columns
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [109]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [110]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_threegram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [111]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Data/3-Talk Time Data Prepare"

Path(path).mkdir(parents=True, exist_ok=True)

In [112]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
331769,oorlogsgezichten,5
331770,opdrachtenlijst,5
331771,verlsaafde,5
331772,oxidatieproces,5


In [113]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [114]:
df_word_select

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
195,genoeg,353128
196,wist,352948
197,oh,351988
198,klaar,350506


In [115]:
df_word_select.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word.xlsx", index=False)

In [116]:
word_list = df_word_select["word"].to_list()
len(word_list)

200

In [117]:
df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
df_twogram_sent

,twogram,frequency
0,kom op,184578
1,dank je,127178
2,dank u,64014
3,wacht even,63826
4,echt waar,59238
...,...,...
328541,zesentwintig horizontaal,3
328542,volledig gedisciplineerd,3
328543,volledig geconcentreerd,3
328544,hero netegengif,3


In [118]:
# Twogram Func
d_list2  = df_twogram_sent.iloc[:,0].values.tolist()

resultlist2 = []
manager = multiprocessing.Manager()
resultlist2 = manager.list()

def word_in_wordgroup(d_list2):
    mergelist2 = []
    try:
        word = d_list2.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist2.append(word[j])
            if len(mergelist2) == len(word):
                    resultlist2.append(d_list2)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list2) # string_word liste

result_list2 = list(resultlist2)
df_result2 = pd.DataFrame(result_list2, columns=[0])  # add columns parameter for empty result
df_result2 = df_result2.rename(columns = {0: "twogram"})
df_merge2 = pd.merge(df_result2, df_twogram_sent, how="left", on="twogram")
df_merge_result2 = df_merge2.sort_values(by="frequency", ascending=False)
df_merge_result2.drop_duplicates(inplace=True)
df_merge_result2.reset_index(drop=True, inplace=True)
df_twogram_select = df_merge_result2
df_twogram_select

,twogram,frequency
0,kom op,184578
1,dank je,127178
2,dank u,64014
3,wacht even,63826
4,echt waar,59238
...,...,...
6564,erg zoals,3
6565,wel veel,3
6566,iedereen doet,3
6567,zal dat,3


In [119]:
df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
df_threegram_sent

,threegram,frequency
0,het spijt me,132758
1,wat is er,106497
2,ik weet het,67518
3,wat is dat,62768
4,wat doe je,60323
...,...,...
550436,vliegen onbekend aantal,3
550437,vliegen obstakel afnemen,3
550438,voor achtduizend euro,3
550439,voor de kijkcijfers,3


In [120]:
# Threegram Func
d_list3  = df_threegram_sent.iloc[:,0].values.tolist()

resultlist3 = []
manager = multiprocessing.Manager()
resultlist3 = manager.list()

def word_in_wordgroup(d_list3):
    mergelist3 = []
    try:
        word = d_list3.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist3.append(word[j])
            if len(mergelist3) == len(word):
                    resultlist3.append(d_list3)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list3) # string_word liste

result_list3 = list(resultlist3)
df_result3 = pd.DataFrame(result_list3, columns=[0])  # add columns parameter for empty result
df_result3 = df_result3.rename(columns = {0: "threegram"})
df_merge3 = pd.merge(df_result3, df_threegram_sent, how="left", on="threegram")
df_merge_result3 = df_merge3.sort_values(by="frequency", ascending=False)
df_merge_result3.drop_duplicates(inplace=True)
df_merge_result3.reset_index(drop=True, inplace=True)
df_threegram_select = df_merge_result3
df_threegram_select

,threegram,frequency
0,het spijt me,132758
1,wat is er,106497
2,ik weet het,67518
3,wat is dat,62768
4,wat doe je,60323
...,...,...
29304,nee het huis,3
29305,ok maar dan,3
29306,niet in jou,3
29307,kijk wij allemaal,3


In [121]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.000,00:00:02.501,dit is de pangale v,BoaYsdPtJYA
1,00:00:04.501,00:00:05.434,unreal,BoaYsdPtJYA
2,00:00:08.518,00:00:13.066,dit is de eerste keer ik rijd in een supersport,BoaYsdPtJYA
3,00:00:13.066,00:00:14.316,ik heb dit in mijn hele leven,BoaYsdPtJYA
4,00:00:14.316,00:00:15.889,nog nooit eerder gedaan,BoaYsdPtJYA
...,...,...,...,...
252485,00:01:37.135,00:01:47.040,soms zitten de volwassenen met natte ogen te k...,MZUDbceIw4E
252486,00:01:47.040,00:01:49.340,dit ziet er al was super indrukwekkend uit en,MZUDbceIw4E
252487,00:01:49.340,00:01:53.600,het is nog maar een voorproefje dus kom zeker ...,MZUDbceIw4E
252488,00:01:53.600,00:01:57.914,tijdsblok te kiezen en zeker ook op tijd te ko...,MZUDbceIw4E


In [122]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [123]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,0.000,2.501,dit is de pangale v,BoaYsdPtJYA
1,4.501,5.434,unreal,BoaYsdPtJYA
2,8.518,13.066,dit is de eerste keer ik rijd in een supersport,BoaYsdPtJYA
3,13.066,14.316,ik heb dit in mijn hele leven,BoaYsdPtJYA
4,14.316,15.889,nog nooit eerder gedaan,BoaYsdPtJYA
...,...,...,...,...
252485,97.135,107.040,soms zitten de volwassenen met natte ogen te k...,MZUDbceIw4E
252486,107.040,109.340,dit ziet er al was super indrukwekkend uit en,MZUDbceIw4E
252487,109.340,113.600,het is nog maar een voorproefje dus kom zeker ...,MZUDbceIw4E
252488,113.600,117.914,tijdsblok te kiezen en zeker ook op tijd te ko...,MZUDbceIw4E


In [124]:
df_sentence_link = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Result/2-Adjust Word Group In Youtube Sentence Word Usage Analysis/{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample_Selected.xlsx")
df_sentence_link = df_sentence_link.iloc[:,[0,1,2,3,4,5]]
df_sentence_link

,search_string,start_time,end_time,sentence,video_id,video_url
0,ik ga u in,30,31,ik ga u in drie stappen uitleggen hoe u de tec,VLM6zkuzvt0,https://www.youtube.com/watch?v=VLM6zkuzvt0&t=30s
1,ik u om uw,66,68,adviseer ik u om uw telefoon erbij te,F0uYZ4uI47w,https://www.youtube.com/watch?v=F0uYZ4uI47w&t=66s
2,u dat u de,292,294,denkt u dat u de enige bent,flv8b2UzJaY,https://www.youtube.com/watch?v=flv8b2UzJaY&t=...
3,ik ga u de,242,243,ik ga u de volgende keer weer zien,NGL5pzrcRKI,https://www.youtube.com/watch?v=NGL5pzrcRKI&t=...
4,hebben hem toen kunnen,51,53,gekomen en kwam de molen buiten werking mijn o...,0rYa2WvEAEo,https://www.youtube.com/watch?v=0rYa2WvEAEo&t=51s
5,gewoon geen genoeg geld,129,130,maar we krijgen gewoon geen genoeg geld,UvDi1HCeLpc,https://www.youtube.com/watch?v=UvDi1HCeLpc&t=...
6,waar mensen door kunnen,145,148,een abonneer button en drie videos waar mensen...,JSDirXX40cU,https://www.youtube.com/watch?v=JSDirXX40cU&t=...
7,mensen hebben echt geen,298,301,van ons want mensen hebben echt geen idee wat ...,PltmSE2Gjvg,https://www.youtube.com/watch?v=PltmSE2Gjvg&t=...
8,niets anders gedaan moet,460,462,ik heb echter niets anders gedaan moet geluk z...,4YSldeXmuXw,https://www.youtube.com/watch?v=4YSldeXmuXw&t=...
9,hebben moeten doen uit onze,86,89,dat we constant grepen hebben moeten doen uit ...,UvDi1HCeLpc,https://www.youtube.com/watch?v=UvDi1HCeLpc&t=86s


In [125]:
twogram_set = set()
threegram_set = set()
for sent in df_sentence_link["search_string"]:
    twogram_zip = ngrams(sent.split(), 2)
    for two in twogram_zip:
        twogram = " ".join(two)
        twogram_set.add(f"{twogram}") 
    threegram_zip = ngrams(sent.split(), 3)
    for three in threegram_zip:
        threegram = " ".join(three)
        threegram_set.add(f"{threegram}")

In [126]:
len(twogram_set)

36

In [127]:
len(threegram_set)

26

In [128]:
twogram_list = list(twogram_set)
#twogram_list

In [129]:
threegram_list = list(threegram_set)
#threegram_list

In [130]:
df_twogram_for_link = df_twogram_select[df_twogram_select["twogram"].isin(twogram_list)]
df_twogram_for_link.reset_index(drop=True, inplace=True)
df_twogram_for_link

,twogram,frequency
0,ik ga,8296
1,niets anders,389
2,willen jullie,90
3,zoals jullie,52
4,jullie hebben,47
5,jullie weten,16
6,wie iedereen,12
7,hebben hem,12
8,ik u,7
9,dat u,5


In [131]:
df_twogram_selected_for_link = word_usage_result(word_list, df_twogram_for_link, "twogram", "frequency", min_word_use_num, max_word_use_num)
df_twogram_selected_for_link 

,twogram,frequency
0,ik ga,8296
1,niets anders,389
2,willen jullie,90
3,zoals jullie,52
4,jullie hebben,47
5,jullie weten,16
6,wie iedereen,12
7,hebben hem,12
8,ik u,7
9,dat u,5


In [132]:
df_threegram_for_link = df_threegram_select[df_threegram_select["threegram"].isin(threegram_list)]
df_threegram_for_link.reset_index(drop=True, inplace=True)
df_threegram_for_link

,threegram,frequency


In [133]:
df_threegram_selected_for_link = word_usage_result(word_list, df_threegram_for_link, "threegram", "frequency", min_word_use_num, max_word_use_num)
df_threegram_selected_for_link 

,threegram,frequency


In [134]:
df_twogram_search_result = word_group_youtube(df_youtube_sentence, df_twogram_selected_for_link["twogram"], "sentence", sample_num)
df_twogram_search_result

,search_string,start_time,end_time,sentence,video_id
0,ik ga,145.75,152.39,en ik ga dus alleen de benedenverdieping schoo...,eiwvySwkjrI
1,niets anders,1863.53,1868.08,als de spoorbeheerder deze niet vindt dan zit ...,UH5OpEpuBCA
2,willen jullie,236.32,244.30,deze hebben de stichting mogelijk al zon twint...,Qw9LiyoG5rI
3,zoals jullie,399.80,404.37,test zoals jullie zien is wel een hele drukke ...,4hMVxIPix0I
4,jullie hebben,2869.73,2874.80,nóg meer mensen bereiken met voedselpakketten ...,EZMgSCfbiGs
5,jullie weten,614.28,620.04,heb jij wat gehoord willen jullie weten wieik ...,UbIX71sXnSY
6,wie iedereen,610.72,614.16,willen jullie weten wie iedereen roept ja,UbIX71sXnSY
7,hebben hem,666.39,669.91,ervan vinden een aantal mensen hebben hem al g...,B2cCnRVFU9g
8,ik u,75.68,78.90,graag verwijs ik u dan naar dit filmpje,w14nQGM3Sqw
9,dat u,2455.73,2456.08,dan stel ik voor dat u verdergaat,k1up2KLkuaM


In [135]:
df_word_group_time_loc_twogram_result = word_group_time_loc(df_twogram_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_twogram_result

,search_string,start_time,end_time,sentence,video_id
0,ik ga,145.899213,146.421461,en ik ga dus alleen de benedenverdieping schoo...,eiwvySwkjrI
1,niets anders,1865.903913,1866.596304,als de spoorbeheerder deze niet vindt dan zit ...,UH5OpEpuBCA
2,willen jullie,242.920741,244.300000,deze hebben de stichting mogelijk al zon twint...,Qw9LiyoG5rI
3,zoals jullie,399.998696,400.694130,test zoals jullie zien is wel een hele drukke ...,4hMVxIPix0I
4,jullie hebben,2873.358529,2874.104118,nóg meer mensen bereiken met voedselpakketten ...,EZMgSCfbiGs
5,jullie weten,616.360000,617.480000,heb jij wat gehoord willen jullie weten wieik ...,UbIX71sXnSY
6,wie iedereen,612.314146,613.488780,willen jullie weten wie iedereen roept ja,UbIX71sXnSY
7,hebben hem,667.677805,668.192927,ervan vinden een aantal mensen hebben hem al g...,B2cCnRVFU9g
8,ik u,76.753333,77.248718,graag verwijs ik u dan naar dit filmpje,w14nQGM3Sqw
9,dat u,2455.899697,2455.973939,dan stel ik voor dat u verdergaat,k1up2KLkuaM


In [136]:
df_word_group_time_loc_twogram_result.start_time = df_word_group_time_loc_twogram_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_twogram_result.end_time = df_word_group_time_loc_twogram_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_twogram_result

,search_string,start_time,end_time,sentence,video_id
0,ik ga,145.299213,147.021461,en ik ga dus alleen de benedenverdieping schoo...,eiwvySwkjrI
1,niets anders,1865.303913,1867.196304,als de spoorbeheerder deze niet vindt dan zit ...,UH5OpEpuBCA
2,willen jullie,242.320741,244.900000,deze hebben de stichting mogelijk al zon twint...,Qw9LiyoG5rI
3,zoals jullie,399.398696,401.294130,test zoals jullie zien is wel een hele drukke ...,4hMVxIPix0I
4,jullie hebben,2872.758529,2874.704118,nóg meer mensen bereiken met voedselpakketten ...,EZMgSCfbiGs
5,jullie weten,615.760000,618.080000,heb jij wat gehoord willen jullie weten wieik ...,UbIX71sXnSY
6,wie iedereen,611.714146,614.088780,willen jullie weten wie iedereen roept ja,UbIX71sXnSY
7,hebben hem,667.077805,668.792927,ervan vinden een aantal mensen hebben hem al g...,B2cCnRVFU9g
8,ik u,76.153333,77.848718,graag verwijs ik u dan naar dit filmpje,w14nQGM3Sqw
9,dat u,2455.299697,2456.573939,dan stel ik voor dat u verdergaat,k1up2KLkuaM


In [137]:
df_word_group_time_loc_twogram_result.start_time = df_word_group_time_loc_twogram_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_twogram_result.end_time = df_word_group_time_loc_twogram_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_twogram_result 

,search_string,start_time,end_time,sentence,video_id
0,ik ga,145,147,en ik ga dus alleen de benedenverdieping schoo...,eiwvySwkjrI
1,niets anders,1865,1867,als de spoorbeheerder deze niet vindt dan zit ...,UH5OpEpuBCA
2,willen jullie,242,245,deze hebben de stichting mogelijk al zon twint...,Qw9LiyoG5rI
3,zoals jullie,399,401,test zoals jullie zien is wel een hele drukke ...,4hMVxIPix0I
4,jullie hebben,2873,2875,nóg meer mensen bereiken met voedselpakketten ...,EZMgSCfbiGs
5,jullie weten,616,618,heb jij wat gehoord willen jullie weten wieik ...,UbIX71sXnSY
6,wie iedereen,612,614,willen jullie weten wie iedereen roept ja,UbIX71sXnSY
7,hebben hem,667,669,ervan vinden een aantal mensen hebben hem al g...,B2cCnRVFU9g
8,ik u,76,78,graag verwijs ik u dan naar dit filmpje,w14nQGM3Sqw
9,dat u,2455,2457,dan stel ik voor dat u verdergaat,k1up2KLkuaM


In [138]:
df_word_group_time_loc_twogram_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_twogram_result['video_id'].map(str)+"&t="+df_word_group_time_loc_twogram_result['start_time'].map(str)+"s"
df_word_group_time_loc_twogram_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,ik ga,145,147,en ik ga dus alleen de benedenverdieping schoo...,eiwvySwkjrI,https://www.youtube.com/watch?v=eiwvySwkjrI&t=...
1,niets anders,1865,1867,als de spoorbeheerder deze niet vindt dan zit ...,UH5OpEpuBCA,https://www.youtube.com/watch?v=UH5OpEpuBCA&t=...
2,willen jullie,242,245,deze hebben de stichting mogelijk al zon twint...,Qw9LiyoG5rI,https://www.youtube.com/watch?v=Qw9LiyoG5rI&t=...
3,zoals jullie,399,401,test zoals jullie zien is wel een hele drukke ...,4hMVxIPix0I,https://www.youtube.com/watch?v=4hMVxIPix0I&t=...
4,jullie hebben,2873,2875,nóg meer mensen bereiken met voedselpakketten ...,EZMgSCfbiGs,https://www.youtube.com/watch?v=EZMgSCfbiGs&t=...
5,jullie weten,616,618,heb jij wat gehoord willen jullie weten wieik ...,UbIX71sXnSY,https://www.youtube.com/watch?v=UbIX71sXnSY&t=...
6,wie iedereen,612,614,willen jullie weten wie iedereen roept ja,UbIX71sXnSY,https://www.youtube.com/watch?v=UbIX71sXnSY&t=...
7,hebben hem,667,669,ervan vinden een aantal mensen hebben hem al g...,B2cCnRVFU9g,https://www.youtube.com/watch?v=B2cCnRVFU9g&t=...
8,ik u,76,78,graag verwijs ik u dan naar dit filmpje,w14nQGM3Sqw,https://www.youtube.com/watch?v=w14nQGM3Sqw&t=76s
9,dat u,2455,2457,dan stel ik voor dat u verdergaat,k1up2KLkuaM,https://www.youtube.com/watch?v=k1up2KLkuaM&t=...


In [139]:
df_word_group_time_loc_twogram_result.to_excel(f"{lang_folder.capitalize()}_Twogram_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx", index=False) 

In [140]:
df_threegram_search_result = word_group_youtube(df_youtube_sentence, df_threegram_selected_for_link["threegram"], "sentence", sample_num)
df_threegram_search_result

""


In [141]:
df_word_group_time_loc_threegram_result = word_group_time_loc(df_threegram_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_threegram_result

,search_string,start_time,end_time,sentence,video_id


In [142]:
df_word_group_time_loc_threegram_result.start_time = df_word_group_time_loc_threegram_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_threegram_result.end_time = df_word_group_time_loc_threegram_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_threegram_result

,search_string,start_time,end_time,sentence,video_id


In [143]:
df_word_group_time_loc_threegram_result.start_time = df_word_group_time_loc_threegram_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_threegram_result.end_time = df_word_group_time_loc_threegram_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_threegram_result 

,search_string,start_time,end_time,sentence,video_id


In [144]:
df_word_group_time_loc_threegram_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_threegram_result['video_id'].map(str)+"&t="+df_word_group_time_loc_threegram_result['start_time'].map(str)+"s"
df_word_group_time_loc_threegram_result

,search_string,start_time,end_time,sentence,video_id,video_url


In [145]:
df_word_group_time_loc_threegram_result.to_excel(f"{lang_folder.capitalize()}_Threegram_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx", index=False) 

In [146]:
df_word_search_result = word_group_youtube(df_youtube_sentence, word_list, "sentence", sample_num)
df_word_search_result

,search_string,start_time,end_time,sentence,video_id
0,ik,1592.600,1598.780,niet altijd een volle trommel van een van die ...,pr7G73DwPXg
1,je,225.850,230.779,je niet ontzettend dapper zijn om juist die kw...,We_B3W2hn6U
2,het,3290.440,3292.330,pleegzorg mogelijk was in het gezin,UH5OpEpuBCA
3,de,17401.280,17405.080,dat zijn niet de effecten die ontstaan door de...,9uOFhXkzEQQ
4,dat,1621.420,1628.794,een veel zachter gesteente of mineraal dan dat...,VujkLK_iD88
...,...,...,...,...,...
195,genoeg,292.040,296.270,het vaak genoeg over gehad maar schijnbaar is ...,TW98lq_819g
196,wist,859.889,864.959,ik in elk geval niet ik wist het al toen vier ...,nTzWwegwMEU
197,oh,1114.590,1120.140,naar hoeveel berichten ik krijgen veel vragen ...,Yrh7Bmlq7uA
198,klaar,839.680,842.140,en dan zet ik alles klaar voor de tweede schik...,qFmXQ000zP0


In [147]:
df_word_group_time_loc_word_result = word_group_time_loc(df_word_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_word_result

,search_string,start_time,end_time,sentence,video_id
0,ik,1597.612667,1597.887333,niet altijd een volle trommel van een van die ...,pr7G73DwPXg
1,je,225.850000,226.044566,je niet ontzettend dapper zijn om juist die kw...,We_B3W2hn6U
2,het,3291.790000,3292.060000,pleegzorg mogelijk was in het gezin,UH5OpEpuBCA
3,de,17402.076774,17402.321935,dat zijn niet de effecten die ontstaan door de...,9uOFhXkzEQQ
4,dat,1624.160779,1624.487062,een veel zachter gesteente of mineraal dan dat...,VujkLK_iD88
...,...,...,...,...,...
195,genoeg,292.411868,292.783736,het vaak genoeg over gehad maar schijnbaar is ...,TW98lq_819g
196,wist,861.508583,861.931083,ik in elk geval niet ik wist het al toen vier ...,nTzWwegwMEU
197,oh,1117.191562,1117.422812,naar hoeveel berichten ik krijgen veel vragen ...,Yrh7Bmlq7uA
198,klaar,840.614800,840.959200,en dan zet ik alles klaar voor de tweede schik...,qFmXQ000zP0


In [148]:
df_word_group_time_loc_word_result.start_time = df_word_group_time_loc_word_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_word_result.end_time = df_word_group_time_loc_word_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_word_result

,search_string,start_time,end_time,sentence,video_id
0,ik,1597.012667,1598.487333,niet altijd een volle trommel van een van die ...,pr7G73DwPXg
1,je,225.250000,226.644566,je niet ontzettend dapper zijn om juist die kw...,We_B3W2hn6U
2,het,3291.190000,3292.660000,pleegzorg mogelijk was in het gezin,UH5OpEpuBCA
3,de,17401.476774,17402.921935,dat zijn niet de effecten die ontstaan door de...,9uOFhXkzEQQ
4,dat,1623.560779,1625.087062,een veel zachter gesteente of mineraal dan dat...,VujkLK_iD88
...,...,...,...,...,...
195,genoeg,291.811868,293.383736,het vaak genoeg over gehad maar schijnbaar is ...,TW98lq_819g
196,wist,860.908583,862.531083,ik in elk geval niet ik wist het al toen vier ...,nTzWwegwMEU
197,oh,1116.591563,1118.022812,naar hoeveel berichten ik krijgen veel vragen ...,Yrh7Bmlq7uA
198,klaar,840.014800,841.559200,en dan zet ik alles klaar voor de tweede schik...,qFmXQ000zP0


In [149]:
df_word_group_time_loc_word_result.start_time = df_word_group_time_loc_word_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_word_result.end_time = df_word_group_time_loc_word_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_word_result 

,search_string,start_time,end_time,sentence,video_id
0,ik,1597,1598,niet altijd een volle trommel van een van die ...,pr7G73DwPXg
1,je,225,227,je niet ontzettend dapper zijn om juist die kw...,We_B3W2hn6U
2,het,3291,3293,pleegzorg mogelijk was in het gezin,UH5OpEpuBCA
3,de,17401,17403,dat zijn niet de effecten die ontstaan door de...,9uOFhXkzEQQ
4,dat,1624,1625,een veel zachter gesteente of mineraal dan dat...,VujkLK_iD88
...,...,...,...,...,...
195,genoeg,292,293,het vaak genoeg over gehad maar schijnbaar is ...,TW98lq_819g
196,wist,861,863,ik in elk geval niet ik wist het al toen vier ...,nTzWwegwMEU
197,oh,1117,1118,naar hoeveel berichten ik krijgen veel vragen ...,Yrh7Bmlq7uA
198,klaar,840,842,en dan zet ik alles klaar voor de tweede schik...,qFmXQ000zP0


In [150]:
df_word_group_time_loc_word_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_word_result['video_id'].map(str)+"&t="+df_word_group_time_loc_word_result['start_time'].map(str)+"s"
df_word_group_time_loc_word_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,ik,1597,1598,niet altijd een volle trommel van een van die ...,pr7G73DwPXg,https://www.youtube.com/watch?v=pr7G73DwPXg&t=...
1,je,225,227,je niet ontzettend dapper zijn om juist die kw...,We_B3W2hn6U,https://www.youtube.com/watch?v=We_B3W2hn6U&t=...
2,het,3291,3293,pleegzorg mogelijk was in het gezin,UH5OpEpuBCA,https://www.youtube.com/watch?v=UH5OpEpuBCA&t=...
3,de,17401,17403,dat zijn niet de effecten die ontstaan door de...,9uOFhXkzEQQ,https://www.youtube.com/watch?v=9uOFhXkzEQQ&t=...
4,dat,1624,1625,een veel zachter gesteente of mineraal dan dat...,VujkLK_iD88,https://www.youtube.com/watch?v=VujkLK_iD88&t=...
...,...,...,...,...,...,...
195,genoeg,292,293,het vaak genoeg over gehad maar schijnbaar is ...,TW98lq_819g,https://www.youtube.com/watch?v=TW98lq_819g&t=...
196,wist,861,863,ik in elk geval niet ik wist het al toen vier ...,nTzWwegwMEU,https://www.youtube.com/watch?v=nTzWwegwMEU&t=...
197,oh,1117,1118,naar hoeveel berichten ik krijgen veel vragen ...,Yrh7Bmlq7uA,https://www.youtube.com/watch?v=Yrh7Bmlq7uA&t=...
198,klaar,840,842,en dan zet ik alles klaar voor de tweede schik...,qFmXQ000zP0,https://www.youtube.com/watch?v=qFmXQ000zP0&t=...


In [151]:
df_word_group_time_loc_word_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx", index=False)

#### Copy Move And Delete

In [152]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx")
output_file

['Dutch_Twogram_With_200_Word_1_Youtube_0.6s_Timeshift_For_Talk_Time_Result.xlsx',
 'Dutch_200_Word_1_Youtube_0.6s_Timeshift_For_Talk_Time_Result.xlsx',
 'Dutch_Threegram_With_200_Word_1_Youtube_0.6s_Timeshift_For_Talk_Time_Result.xlsx']

In [153]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [154]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass